In [1]:
%matplotlib notebook
from qcodes.instrument_drivers.tektronix.AWG5208 import AWG5208
import broadbean as bb

ramp = bb.PulseAtoms.ramp
sine = bb.PulseAtoms.sine

In [2]:
# First we form the measurement sequence

SR = 1e9
N = 3  # the number of times we repeat the first part
measurement_freqs = [5e4, 1e5, 2e5]

meas_bp_ch1 = bb.BluePrint()
meas_bp_ch1.insertSegment(0, sine, (1e5, 0.1, 0, 0), dur=200e-6)
meas_bp_ch1.insertSegment(-1, ramp, (0, 0), dur=40e-6)
meas_bp_ch1.setSR(SR)

meas_bp_ch2 = bb.BluePrint()
meas_bp_ch2.insertSegment(0, ramp, (0, 0), dur=200e-6)
meas_bp_ch2.insertSegment(-1, ramp, (0.1, 0.1), dur=20e-6)
meas_bp_ch2.insertSegment(-1, ramp, (0, 0), dur=20e-6)
meas_bp_ch2.setSR(SR)

meas_elem = bb.Element()
meas_elem.addBluePrint('signal_channel', meas_bp_ch1)
meas_elem.addBluePrint('trigger_channel', meas_bp_ch2)


reset_bp = bb.BluePrint()
reset_bp.insertSegment(0, ramp, (0, 0.05), dur=120e-6)
reset_bp.insertSegment(1, ramp, (0.05, 0), dur=120e-6)
reset_bp.setSR(SR)

reset_elem = bb.Element()
reset_elem.addBluePrint('signal_channel', reset_bp)
reset_elem.addBluePrint('trigger_channel', reset_bp)


measureseq = bb.Sequence()
measureseq.setSR(SR)
measureseq.addElement(1, meas_elem)
measureseq.addElement(2, reset_elem)
measureseq.setSequencingNumberOfRepetitions(1, N)

measureseq.plotSequence()

C:\ProgramData\Anaconda3\lib\site-packages\broadbean\broadbean.py:72: UserWarning: plotSequence is obsolete. Please use broadbean.plotting.plotter insted.
  warnings.warn(warnstr)


In [ ]:
# And then we turn that into a master sequence

mainseq = bb.Sequence()
mainseq.setSR(SR)

for pos, freq in enumerate(measurement_freqs):
    subseq = measureseq.copy()
    subseq.element(1).changeArg('signal_channel', 'sine',
                                'freq', freq)
    mainseq.addSubSequence(pos+1, subseq)

mainseq.plotSequence()

# The visualisation is not terribly helpful in this case, but
# we see that three subsequences are present

In [ ]:
# Connect

awg = AWG5208('awg', 'TCPIP0::192.168.15.118::inst0::INSTR')

In [ ]:
# forge the sequence
fs = mainseq.forge()

# send it to the instrument
seqname = 'mytestseq'
amplitudes = [ch.awg_amplitude() for ch in awg.channels][:2]
channel_mapping = {'trigger_channel': 2, 'signal_channel': 1}
seqx_file = AWG5208.makeSEQXFileFromForgedSequence(fs,
                                                   amplitudes=amplitudes,
                                                   seqname=seqname,
                                                   channel_mapping=channel_mapping)

# load it and assign its tracks to the channels
filename = 'mainplussub.seqx'

awg.clearSequenceList()
awg.clearWaveformList()
awg.sendSEQXFile(seqx_file, filename=filename)
awg.loadSEQXFile(filename)
awg.ch1.setSequenceTrack(seqname, 1)
awg.ch2.setSequenceTrack(seqname, 2)